In [5]:
# natural language processing
# 1. collect data
# 2. featurize
# 3. compare features

# vectors can be created out of bads of words
# cosine similarity can be used on vectors to determine 
# similarity

# Term Frequency (TR) - 
# import of the term within that document
# TF(d,t) = number of occurences of term t in document d

# Inverse Document Frequency (TR) - 
# importance of the term in the corpus
# IDF(t) = log(D/t) where
# D = total number of documents
# t = number of ducments with the team

In [ ]:
import nltk

In [ ]:
nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


In [ ]:
fpath = '/Users/bcutrell/Python-Data-Science-and-Machine-Learning-Bootcamp/Machine Learning Sections/Natural-Language-Processing/smsspamcollection/SMSSpamCollection'
messages = [ line.rstrip() for line in open(fpath, encoding='utf-8')]

In [ ]:
len(messages)

In [ ]:
messages[50]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
messages = pd.read_csv(fpath, sep='\t', names=['label', 'message'])

In [ ]:
messages.describe()

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages['length'] = messages['message'].apply(len)
messages['length'].plot.hist(bins=100)

In [ ]:
messages['length'].describe()

In [ ]:
messages[messages['length'] == 910]['message'].iloc[0]

In [ ]:
messages.hist(column='length', by='label', bins=60, figsize=(12,4))

In [ ]:
# Spam messages tend to have more characters
# so length appears to be a good feature

In [ ]:
import string

In [ ]:
mess = 'Sample message! Notice: it has punctuation.'

In [ ]:
nopunc = [c for c in mess if c not in string.punctuation]

In [ ]:
from nltk.corpus import stopwords # stopwords.words('en')

In [ ]:
nopunc = ''.join(nopunc)

In [ ]:
clean_mess = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
clean_mess

In [ ]:
def text_process(mess):
    '''
    1. remove punc
    2. remove stop words
    3. return list of clean text words
    '''
    nopunc = [c for c in mess if c not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages.head()

In [ ]:
messages['message'].head(5).apply(text_process)

In [ ]:
# stemming can also be used to combine 
# like words (running, ran, run)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

In [ ]:
mess4 = messages['message'][3]

In [ ]:
print(mess4)

In [ ]:
bow4 = bow_transformer.transform([mess4])

In [ ]:
print(bow4)

In [ ]:
print(bow4.shape)

In [ ]:
bow_transformer.get_feature_names()[4068]

In [ ]:
messages_bow = bow_transformer.transform(messages['message'])

In [ ]:
print('Shape of Sparse Matrix: ', messages_bow.shape)

In [ ]:
messages_bow.nnz

In [ ]:
sparsity = (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))
print('sparsity: {}'.format(sparsity))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [ ]:
tfidf4 = tfidf_transformer.transform(bow4)

In [ ]:
print(tfidf4)

In [ ]:
tfidf_transformer.idf_[bow_transformer.vocabulary_['university']]

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [ ]:
# messages are now numerical vectors

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
spam_detect_model = MultinomialNB().fit(messages_tfidf, messages['label'])

In [ ]:
spam_detect_model.predict(tfidf4)[0]

In [ ]:
messages['label'][3]

In [ ]:
all_pred = spam_detect_model.predict(messages_tfidf)

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X = messages['message']
y = messages['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
# precision (also called positive predictive value) 
# is the fraction of relevant instances among the retrieved instances
# true positives / false positives

# recall (also known as sensitivity) is the fraction of relevant instances 
# that have been retrieved over the total amount of relevant instances.
# true positives / total relevant

In [ ]:
##############################
# Project
##############################

In [ ]:
fpath = '/Users/bcutrell/Python-Data-Science-and-Machine-Learning-Bootcamp/Machine Learning Sections/Natural-Language-Processing/yelp.csv'
yelp = pd.read_csv(messages)